# Linear Regression (Baseline)

## Imports

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pickle


## Lectura de datos

In [6]:
X_train = pd.read_csv("../../data/processed/X_train.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv")["RUL_steps"].values
X_val = pd.read_csv("../../data/processed/X_val.csv")
y_val = pd.read_csv("../../data/processed/y_val.csv")["RUL_steps"].values
train_balanced =  pd.read_csv("../../data/processed/train_balanced.csv")
train_df = pd.read_csv("../../data/processed/train_df.csv")
val_df = pd.read_csv("../../data/processed/val_df.csv")

In [7]:
print("🎯 ENTRENAMIENTO MODELOS LINEALES")

# Lista para métricas totales
modelos = []

# 1. LINEAR REGRESSION SIMPLE
print("Entrenando Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_val)
modelos.append({'Modelo': 'Linear Regression', 
                'MAE_total': mean_absolute_error(y_val, lr_pred), 
                'R2_total': r2_score(y_val, lr_pred)})

# 2. POLYNOMIAL REGRESSION (grado 2)
print("Entrenando Polynomial Regression...")
poly_model = make_pipeline(PolynomialFeatures(degree=2, include_bias=False), LinearRegression())
poly_model.fit(X_train, y_train)
poly_pred = poly_model.predict(X_val)
modelos.append({'Modelo': 'Polynomial (grado 2)', 
                'MAE_total': mean_absolute_error(y_val, poly_pred), 
                'R2_total': r2_score(y_val, poly_pred)})

# 3. RIDGE REGRESSION
print("Entrenando Ridge Regression...")
ridge_model = make_pipeline(StandardScaler(), Ridge(alpha=10.0))
ridge_model.fit(X_train, y_train)
ridge_pred = ridge_model.predict(X_val)
modelos.append({'Modelo': 'Ridge (alpha=10)', 
                'MAE_total': mean_absolute_error(y_val, ridge_pred), 
                'R2_total': r2_score(y_val, ridge_pred)})

# 4. LASSO REGRESSION
print("Entrenando Lasso Regression...")
lasso_model = make_pipeline(StandardScaler(), Lasso(alpha=1.0, max_iter=10000))
lasso_model.fit(X_train, y_train)
lasso_pred = lasso_model.predict(X_val)
modelos.append({'Modelo': 'Lasso (alpha=1)', 
                'MAE_total': mean_absolute_error(y_val, lasso_pred), 
                'R2_total': r2_score(y_val, lasso_pred)})

# TABLA MÉTRICAS TOTALES
resultados_total = pd.DataFrame(modelos).sort_values('MAE_total').reset_index(drop=True)
print("\n📊 MÉTRICAS TOTALES:")
print(resultados_total.round(4))

# MÉTRICAS CASOS CRÍTICOS (RUL < 50)
print("\n🔥 MÉTRICAS RUL<50 (CRÍTICOS):")
mask_critica = y_val < 50
print(f"Nº casos críticos: {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")

mae_criticos = [
    mean_absolute_error(y_val[mask_critica], lr_pred[mask_critica]),
    mean_absolute_error(y_val[mask_critica], poly_pred[mask_critica]),
    mean_absolute_error(y_val[mask_critica], ridge_pred[mask_critica]),
    mean_absolute_error(y_val[mask_critica], lasso_pred[mask_critica])
]

resultados_criticos = pd.DataFrame({
    'Modelo': ['Linear Regression', 'Polynomial (grado 2)', 'Ridge (alpha=10)', 'Lasso (alpha=1)'],
    'MAE_RUL<50': np.round(mae_criticos, 2)
}).sort_values('MAE_RUL<50').reset_index(drop=True)

print(resultados_criticos)

# RESUMEN FINAL
print("\n🏆 RESUMEN:")
print(f"MEJOR MAE total: {resultados_total.iloc[0]['Modelo']} ({resultados_total.iloc[0]['MAE_total']:.2f})")
print(f"MEJOR MAE críticos: {resultados_criticos.iloc[0]['Modelo']} ({resultados_criticos.iloc[0]['MAE_RUL<50']:.2f})")


🎯 ENTRENAMIENTO MODELOS LINEALES
Entrenando Linear Regression...
Entrenando Polynomial Regression...
Entrenando Ridge Regression...
Entrenando Lasso Regression...

📊 MÉTRICAS TOTALES:
                 Modelo  MAE_total  R2_total
0      Ridge (alpha=10)    33.5983   -0.0454
1     Linear Regression    33.5983   -0.0454
2       Lasso (alpha=1)    33.6117   -0.0474
3  Polynomial (grado 2)    33.6495   -0.0440

🔥 MÉTRICAS RUL<50 (CRÍTICOS):
Nº casos críticos: 24,643 (62.1%)
                 Modelo  MAE_RUL<50
0     Linear Regression       19.32
1      Ridge (alpha=10)       19.32
2       Lasso (alpha=1)       19.37
3  Polynomial (grado 2)       19.63

🏆 RESUMEN:
MEJOR MAE total: Ridge (alpha=10) (33.60)
MEJOR MAE críticos: Linear Regression (19.32)


### Creación de pickle

In [8]:
modelos_a_guardar = {
    "lr_model": lr_model,
    "poly_model": poly_model,
    "ridge_model": ridge_model,
    "lasso_model": lasso_model
}

# Guardar cada modelo en un archivo individual
for nombre, objeto in modelos_a_guardar.items():
    ruta = f"../../models/{nombre}.pkl"  
    with open(ruta, "wb") as f:
        pickle.dump(objeto, f)